In [ ]:
!pip3 install datasets
!pip3 install -U packaging setuptools wheel ninja
!pip3 install --no-build-isolation axolotl[flash-attn,deepspeed]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ninja]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      
      
      torch.__version__  = 2.7.1+cpu
      
      
      /tmp/pip-install-nul5_k5m/flash-attn_cfa2c609c0f64687b0e9df5b65095e4d/setup.py:106: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc

In [24]:
from datasets import load_dataset, Image, Dataset
import json, pathlib

src_ds   = (load_dataset("philschmid/amazon-product-descriptions-vlm", split="train")
              .cast_column("image", Image(decode=False)))   

# Let's observe what a row looks like
display(src_ds[0])

{'image': {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe1\x00JExif\x00\x00II*\x00\x08\x00\x00\x00\x01\x00i\x87\x04\x00\x01\x00\x00\x00\x1a\x00\x00\x00\x00\x00\x00\x00\x01\x00\x86\x92\x07\x00\x15\x00\x00\x00,\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Version 1.0.0\x00\xff\xdb\x00C\x00\x0b\x08\x08\x08\x08\x08\x0b\x08\x08\x0b\x10\x0b\t\x0b\x10\x13\x0e\x0b\x0b\x0e\x13\x16\x12\x12\x13\x12\x12\x16\x15\x11\x13\x12\x12\x13\x11\x15\x15\x19\x1a\x1b\x1a\x19\x15!!$$!!0///06666666666\xff\xdb\x00C\x01\x0c\x0b\x0b\x0c\r\x0c\x0f\r\r\x0f\x13\x0e\x0e\x0e\x13\x14\x0e\x0f\x0f\x0e\x14\x1a\x12\x12\x14\x12\x12\x1a"\x18\x15\x15\x15\x15\x18"\x1e \x1b\x1b\x1b \x1e%%""%%//,//6666666666\xff\xc0\x00\x11\x08\x01\xf4\x01\xf4\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x

In [21]:

def to_axolotl(sample):
    prompt = (
        """Create a short product description for ##PRODUCT NAME## in the ##CATEGORY## category. 
        Only output the description and ensure that it is SEO optimized"""
    )
    return {
        "messages": [
            {"role": "system",    "content": [
                {"type": "text", "text": "You are an expert Amazon copy-writer."}
            ]},
            {"role": "user",      "content": [
                {"type": "text",  "text": prompt
                 .replace("##PRODUCT NAME##", sample.get("Product Name", ""))
                 .replace("##CATEGORY##",     str(sample.get("Category", "")))},
                {"type": "image", "path": sample["image"]["path"]}       # PIL.Image is OK
            ]},
            {"role": "assistant", "content": [
                {"type": "text",  "text": sample["description"]}
            ]}
        ]
    }

out_file = pathlib.Path("vlm_train.jsonl").open("w")
for row in src_ds:
    out_file.write(json.dumps(to_axolotl(row), ensure_ascii=False) + "\n")
out_file.close()


In [25]:
to_axolotl(src_ds[0])

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are an expert Amazon copy-writer.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': "Create a short product description for Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue in the Toys & Games | Kids' Electronics | Electronic Learning Toys category. \n        Only output the description and ensure that it is SEO optimized"},
    {'type': 'image', 'path': '002e4642d3ead5ecdc9958ce0b3a5a79.jpg'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': "Kurio Glow Smartwatch: Fun, Safe & Educational!  This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play!  #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"}]}]}